# In this run, the agenda...
clean irrelevant features
word to vec

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [2]:
from catboost import CatBoostRegressor

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load data

In [4]:
df = pd.read_csv('train.csv')
df = df.set_index('id')

In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Basic data cleaning

In [8]:
def clean(df):

    cleaned = df.copy()
    cleaned['transit'] = 1-df['transit'].isna().astype(int) #there is transportation listed
    cleaned['host_is_superhost'] = (df['host_is_superhost'] == 't').astype(int)
    cleaned['host_has_profile_pic'] = (df['host_has_profile_pic'] == 't').astype(int)
    cleaned['host_identity_verified'] = (df['host_identity_verified'] == 't').astype(int)
    cleaned['instant_bookable'] = (df['instant_bookable'] == 't').astype(int)
    cleaned['is_business_travel_ready'] = (df['is_business_travel_ready'] == 't').astype(int)
    cleaned['require_guest_profile_picture'] = (df['require_guest_profile_picture'] == 't').astype(int)
    cleaned['require_guest_phone_verification'] = (df['require_guest_phone_verification'] == 't').astype(int)
    cleaned['extra_people'] = df['extra_people'].str.replace('$', '').astype('float')

    #more complicated cleaning
    
    #impute missing values
    num_features_w_missing = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                          'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                          'review_scores_value', 'host_listings_count', 'bedrooms', 'bathrooms', 
                          'square_feet' ]
    ki = KNNImputer()
    t = ki.fit_transform(cleaned[num_features_w_missing])
    
    std = StandardScaler()
    std.fit_transform(t)
    
    cleaned[num_features_w_missing] = t
    
    #standardize all numerical features

    #clean amenities
    def clean_amenities(x):
        return x.strip('{').strip('}').replace('"','').split(',')
    cleaned['amenities'] = df['amenities'].apply(clean_amenities)
    mlb = MultiLabelBinarizer()
    cleaned[mlb.classes_] = mlb.fit_transform(cleaned['amenities'])

    #one hot encode bed type
    bed_type_ohe = OneHotEncoder()
    bed_type_fit = bed_type_ohe.fit_transform(df[['bed_type']])
    cleaned[bed_type_ohe.categories_[0]] = bed_type_fit.toarray() #stackoverflow诚不欺我

    #rate cancellation policy
    dict_cancellation_policy = {'flexible': 1, 'moderate': 2, 'super_strict_60': 7,
       'strict_14_with_grace_period': 5, 'super_strict_30': 6, 'long_term': 4,
       'strict': 3} #reference: https://www.airbnb.com/help/article/475
    cleaned['cancellation_policy'] = df['cancellation_policy'].apply(lambda x: dict_cancellation_policy[x])

    # host response time
    dict_response_time = {np.NaN: 0, 'within an hour': 1, 'within a few hours': 2, 'within a day': 3,'a few days or more': 4}
    cleaned['host_response_time'] = df['host_response_time'].apply(lambda x: dict_response_time[x])

    #last review first review difference
    time_diff = pd.to_datetime(df['last_review']) - pd.to_datetime(df['first_review'])
    cleaned['time_diff'] = time_diff.dt.days

    #host history
    first_date = min(pd.to_datetime(df['host_since']))
    cleaned['host history'] = (pd.to_datetime(df['host_since']) - first_date).dt.days

    mlb_v = MultiLabelBinarizer()
    cleaned[mlb_v.classes_] = mlb_v.fit_transform(df['host_verifications'].str.replace('[', '').str.replace(']', '').str.replace('\'', '').str.replace(' ', '').str.split(','))

    #neighborhood encode bed type
    neighbor_ohe = OneHotEncoder()
    neighbor_fit = neighbor_ohe.fit_transform(df[['neighbourhood_group_cleansed']])
    cleaned[neighbor_ohe.categories_[0]] = neighbor_fit.toarray() 

    #market encode bed type
    market_ohe = OneHotEncoder()
    market_fit = market_ohe.fit_transform(df[['market']])
    cleaned[market_ohe.categories_[0]] = market_fit.toarray() 

    cleaned['country'] = (df['country_code'] == 'US').astype(int)

    #property encode bed type
    property_ohe = OneHotEncoder()
    property_fit = property_ohe.fit_transform(df[['property_type']])
    cleaned[property_ohe.categories_[0]] = property_fit.toarray() 

    #room type encode bed type
    room_ohe = OneHotEncoder()
    room_fit = room_ohe.fit_transform(df[['room_type']])
    cleaned[room_ohe.categories_[0]] = room_fit.toarray() 

    #host_response_rate
    cleaned['host_response_rate'] = df['host_response_rate'].str.strip('%').astype(float)

    string_cols = ['name', 'summary', 'space', 'description', 'experiences_offered', 'host_acceptance_rate',
       'neighborhood_overview', 'notes', 'access','interaction', 'house_rules', 'host_id', 'host_name',
               'host_location', 'host_acceptance_rate', 'host_response_rate', 'host_about', 'host_neighbourhood', 'host_since',
     '','host_verifications', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'neighbourhood_cleansed',  
               'city', 'bed_type', 'amenities', 'state', 'zipcode', 'market', 'country_code', 'first_review', 'last_review']

    X2 = cleaned.drop(string_cols, axis=1)
    return X2


In [9]:
cleaned = clean(df)

# Get correlations and identify obslete features

In [81]:
# np.array(cleaned.columns)
X4 = cleaned
y4 = cleaned['price']

In [82]:
X4_extra_features = ['Kyoto',
 'Adirondacks',
 'Chalet',
 'Paris',
 'Boston',
 'Island',
 'Jamaica South Coast',
 'Timeshare',
 'Agra',
 'Pool with pool hoist',
 'Catskills and Hudson Valley',
 'Houseboat',
 'Los Angeles',
 'San Francisco',
 'New Orleans',
 'Train']

In [83]:
X4 = X4.drop(X4_extra_features, axis=1)

In [84]:
feature_corr = np.abs(X4[X4.columns].corr()['price'])

In [85]:
feature_corr = np.abs(X4[X4.columns].corr()['price'])
feature_corr_values = feature_corr.sort_values()
feature_weight_df = pd.DataFrame(feature_corr_values)
small_1 = feature_corr_values < 0.05

## Try deleting features with a correlation smaller than 0.1

In [86]:
small_1 = feature_corr_values < 0.05

In [87]:
small_1_to_delete = list(small_1[small_1 == True].index) + ['is_business_travel_ready', 'price']

In [77]:
feature_weight_001 = feature_weight_df[feature_weight_df['price'] < 0.05].to_xarray()

In [88]:
# to drop before train
X4 = X4.drop(small_1_to_delete, axis=1)

In [89]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X4, y4, test_size=0.25, shuffle=True)

In [96]:
params = {'max_depth': range(1, 20, 1),
         'min_child_weight': range(1, 10, 1),
          'eta': np.arange(0.01, 0.2, 0.02),
         }

In [98]:
rv = RandomizedSearchCV(xgb, params)
rv.fit(X4, y4)
rv.best_estimator_

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.16999999999999998,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [101]:
from xgboost import XGBRegressor
xgb = XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.16999999999999998,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None)

In [102]:
xgb.fit(X2_train, y2_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.16999999999999998,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [103]:
# train accuracy - RMSE
np.sqrt(sum((xgb.predict(X2_train) - y2_train)**2) / len(y2_train))

73.9364905780139

In [104]:
# val accuracy
np.sqrt(sum((xgb.predict(X2_val) - y2_val)**2) / len(y2_val))

88.93877384550278

## Try CatRegressor

In [54]:
cat = CatBoostRegressor()
cat.fit(X2_train, y2_train);

Learning rate set to 0.068151
0:	learn: 129.0367774	total: 69.3ms	remaining: 1m 9s
1:	learn: 125.7472505	total: 73.5ms	remaining: 36.7s
2:	learn: 122.6990026	total: 76.9ms	remaining: 25.5s
3:	learn: 120.1998554	total: 79.9ms	remaining: 19.9s
4:	learn: 117.6923033	total: 83.5ms	remaining: 16.6s
5:	learn: 115.4868926	total: 86.9ms	remaining: 14.4s
6:	learn: 113.5427107	total: 90ms	remaining: 12.8s
7:	learn: 111.8313276	total: 93.1ms	remaining: 11.5s
8:	learn: 110.2757582	total: 96.2ms	remaining: 10.6s
9:	learn: 108.8252577	total: 99.4ms	remaining: 9.84s
10:	learn: 107.5381035	total: 103ms	remaining: 9.23s
11:	learn: 106.4178018	total: 106ms	remaining: 8.72s
12:	learn: 105.3596263	total: 109ms	remaining: 8.28s
13:	learn: 104.4413457	total: 113ms	remaining: 7.96s
14:	learn: 103.5659086	total: 116ms	remaining: 7.62s
15:	learn: 102.7947588	total: 119ms	remaining: 7.32s
16:	learn: 102.1097881	total: 122ms	remaining: 7.06s
17:	learn: 101.3838891	total: 125ms	remaining: 6.83s
18:	learn: 100.849

160:	learn: 88.3867240	total: 585ms	remaining: 3.05s
161:	learn: 88.3745342	total: 588ms	remaining: 3.04s
162:	learn: 88.3099844	total: 591ms	remaining: 3.04s
163:	learn: 88.2885960	total: 594ms	remaining: 3.03s
164:	learn: 88.2410265	total: 598ms	remaining: 3.03s
165:	learn: 88.2043528	total: 601ms	remaining: 3.02s
166:	learn: 88.1550813	total: 604ms	remaining: 3.01s
167:	learn: 88.1411421	total: 608ms	remaining: 3.01s
168:	learn: 88.1233475	total: 612ms	remaining: 3.01s
169:	learn: 88.0844810	total: 616ms	remaining: 3.01s
170:	learn: 88.0497405	total: 619ms	remaining: 3s
171:	learn: 88.0052497	total: 623ms	remaining: 3s
172:	learn: 87.9752830	total: 626ms	remaining: 2.99s
173:	learn: 87.9322933	total: 630ms	remaining: 2.99s
174:	learn: 87.8830994	total: 633ms	remaining: 2.98s
175:	learn: 87.8643723	total: 637ms	remaining: 2.98s
176:	learn: 87.8193280	total: 640ms	remaining: 2.98s
177:	learn: 87.7739904	total: 644ms	remaining: 2.97s
178:	learn: 87.7379941	total: 647ms	remaining: 2.97s

338:	learn: 84.0394256	total: 1.18s	remaining: 2.3s
339:	learn: 84.0195745	total: 1.18s	remaining: 2.3s
340:	learn: 84.0063873	total: 1.19s	remaining: 2.3s
341:	learn: 83.9869802	total: 1.19s	remaining: 2.29s
342:	learn: 83.9551129	total: 1.2s	remaining: 2.3s
343:	learn: 83.9398934	total: 1.2s	remaining: 2.3s
344:	learn: 83.9141931	total: 1.21s	remaining: 2.29s
345:	learn: 83.9054562	total: 1.21s	remaining: 2.29s
346:	learn: 83.8824726	total: 1.22s	remaining: 2.3s
347:	learn: 83.8711121	total: 1.23s	remaining: 2.3s
348:	learn: 83.8598448	total: 1.23s	remaining: 2.3s
349:	learn: 83.8269235	total: 1.24s	remaining: 2.3s
350:	learn: 83.8118860	total: 1.24s	remaining: 2.3s
351:	learn: 83.7981506	total: 1.25s	remaining: 2.29s
352:	learn: 83.7842813	total: 1.25s	remaining: 2.29s
353:	learn: 83.7657282	total: 1.25s	remaining: 2.28s
354:	learn: 83.7524369	total: 1.25s	remaining: 2.28s
355:	learn: 83.7319425	total: 1.26s	remaining: 2.27s
356:	learn: 83.7058712	total: 1.26s	remaining: 2.27s
357:	

511:	learn: 81.5685466	total: 1.78s	remaining: 1.69s
512:	learn: 81.5565818	total: 1.78s	remaining: 1.69s
513:	learn: 81.5386897	total: 1.78s	remaining: 1.69s
514:	learn: 81.5235583	total: 1.79s	remaining: 1.69s
515:	learn: 81.5142295	total: 1.79s	remaining: 1.68s
516:	learn: 81.4986322	total: 1.8s	remaining: 1.68s
517:	learn: 81.4793909	total: 1.8s	remaining: 1.67s
518:	learn: 81.4501936	total: 1.8s	remaining: 1.67s
519:	learn: 81.4373643	total: 1.81s	remaining: 1.67s
520:	learn: 81.4292497	total: 1.81s	remaining: 1.66s
521:	learn: 81.4143469	total: 1.81s	remaining: 1.66s
522:	learn: 81.4001406	total: 1.82s	remaining: 1.66s
523:	learn: 81.3835849	total: 1.82s	remaining: 1.65s
524:	learn: 81.3630298	total: 1.82s	remaining: 1.65s
525:	learn: 81.3518292	total: 1.83s	remaining: 1.65s
526:	learn: 81.3351014	total: 1.83s	remaining: 1.64s
527:	learn: 81.3210778	total: 1.83s	remaining: 1.64s
528:	learn: 81.3073986	total: 1.84s	remaining: 1.64s
529:	learn: 81.2985546	total: 1.84s	remaining: 1.

694:	learn: 79.3286847	total: 2.37s	remaining: 1.04s
695:	learn: 79.2931913	total: 2.38s	remaining: 1.04s
696:	learn: 79.2857517	total: 2.38s	remaining: 1.03s
697:	learn: 79.2628672	total: 2.38s	remaining: 1.03s
698:	learn: 79.2521121	total: 2.39s	remaining: 1.03s
699:	learn: 79.2458995	total: 2.39s	remaining: 1.02s
700:	learn: 79.2374734	total: 2.4s	remaining: 1.02s
701:	learn: 79.2342140	total: 2.4s	remaining: 1.02s
702:	learn: 79.2176428	total: 2.4s	remaining: 1.01s
703:	learn: 79.2042279	total: 2.4s	remaining: 1.01s
704:	learn: 79.1957543	total: 2.41s	remaining: 1.01s
705:	learn: 79.1889015	total: 2.41s	remaining: 1s
706:	learn: 79.1864418	total: 2.42s	remaining: 1s
707:	learn: 79.1736139	total: 2.42s	remaining: 998ms
708:	learn: 79.1587704	total: 2.42s	remaining: 994ms
709:	learn: 79.1470909	total: 2.42s	remaining: 991ms
710:	learn: 79.1415670	total: 2.43s	remaining: 987ms
711:	learn: 79.1293812	total: 2.43s	remaining: 983ms
712:	learn: 79.1180783	total: 2.43s	remaining: 980ms
713

880:	learn: 77.5856782	total: 2.97s	remaining: 402ms
881:	learn: 77.5739827	total: 2.98s	remaining: 398ms
882:	learn: 77.5687493	total: 2.98s	remaining: 395ms
883:	learn: 77.5512825	total: 2.98s	remaining: 391ms
884:	learn: 77.5434313	total: 2.99s	remaining: 388ms
885:	learn: 77.5327383	total: 2.99s	remaining: 385ms
886:	learn: 77.5160980	total: 2.99s	remaining: 381ms
887:	learn: 77.4969786	total: 3s	remaining: 378ms
888:	learn: 77.4847890	total: 3s	remaining: 375ms
889:	learn: 77.4758462	total: 3s	remaining: 371ms
890:	learn: 77.4650017	total: 3.01s	remaining: 368ms
891:	learn: 77.4544542	total: 3.01s	remaining: 365ms
892:	learn: 77.4521623	total: 3.01s	remaining: 361ms
893:	learn: 77.4372659	total: 3.02s	remaining: 358ms
894:	learn: 77.4327132	total: 3.02s	remaining: 354ms
895:	learn: 77.4294539	total: 3.02s	remaining: 351ms
896:	learn: 77.4149239	total: 3.03s	remaining: 348ms
897:	learn: 77.4075387	total: 3.03s	remaining: 344ms
898:	learn: 77.3916605	total: 3.03s	remaining: 341ms
89

In [55]:
# train accuracy - RMSE
np.sqrt(sum((cat.predict(X2_train) - y2_train)**2) / len(y2_train))

76.51043292473841

In [56]:
# val accuracy
np.sqrt(sum((cat.predict(X2_val) - y2_val)**2) / len(y2_val))

89.08356553490037

## Load test data & clean

In [112]:
test = pd.read_csv('test.csv')
test = test.set_index('id')

In [113]:
cleaned_test = clean(test)

# Train new predictor based on the entire df - XGB

In [105]:
xgb = XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.16999999999999998,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None)
xgb.fit(X4, y4)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.16999999999999998,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [106]:
# train accuracy - RMSE
np.sqrt(sum((xgb.predict(X4) - y4)**2) / len(y4))

74.95865119753961

In [116]:
test_extra_features = list(set(cleaned_test.columns) - set(X2.columns) - set('price')) #columns in test that are not in X2
test_extra_features

['South Bay, CA',
 'D.C.',
 'Atlanta',
 'Ceiling hoist',
 'Kitchenette',
 'Cuba',
 'photographer',
 'Castle',
 'Lagos, NG',
 'Nature lodge',
 'Cave',
 'Earth house',
 'London']

In [117]:
cleaned_test = cleaned_test.drop(test_extra_features, axis=1)

In [111]:
cleaned_test.columns

Index(['accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet',
       'guests_included', 'review_scores_location', 'cancellation_policy',
       'Air conditioning', 'Building staff', 'Cable TV', 'Crib', 'Dishwasher',
       'Doorman', 'Dryer', 'Elevator', 'Family/kid friendly', 'Gym',
       'Indoor fireplace', 'Lock on bedroom door', 'Pack ’n Play/travel crib',
       'Paid parking off premises', 'TV', 'Washer',
       'translation missing: en.hosting_amenity_49',
       'translation missing: en.hosting_amenity_50', 'Brooklyn', 'Manhattan',
       'Queens', 'Serviced apartment', 'Entire home/apt', 'Private room'],
      dtype='object')

In [110]:
X4.columns

Index(['host_listings_count', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'square_feet', 'guests_included', 'extra_people',
       'review_scores_location', 'cancellation_policy',
       'calculated_host_listings_count', '24-hour check-in',
       'Air conditioning', 'Babysitter recommendations', 'Bathtub',
       'Building staff', 'Cable TV', 'Cat(s)', 'Children’s books and toys',
       'Coffee maker', 'Crib', 'Dishwasher', 'Doorman', 'Dryer', 'Elevator',
       'Family/kid friendly', 'Fire extinguisher', 'Free street parking',
       'Gym', 'Hair dryer', 'High chair', 'Indoor fireplace', 'Internet',
       'Iron', 'Kitchen', 'Laptop friendly workspace', 'Lock on bedroom door',
       'Long term stays allowed', 'Pack ’n Play/travel crib',
       'Paid parking off premises', 'Paid parking on premises',
       'Patio or balcony', 'Pets allowed', 'Pets live on this property',
       'Pool', 'Private entrance', 'Room-darkening shades', 'Safety card',
       'Self check-in', '

In [118]:
#reorder the features
cleaned_test = cleaned_test[X4.columns]

In [119]:
test_pred = xgb.predict(cleaned_test)

In [122]:
pd.DataFrame(index=test.index, data=test_pred, columns=['Predicted']).to_csv('preds4-0005.csv')

# Train new predictor based on the entire df - cat

In [136]:
cat = CatBoostRegressor()
cat.fit(X2_entire_train, y2)
cat_test_pred = cat.predict(cleaned_test)
pd.DataFrame(index=test.index, data=test_pred, columns=['Predicted']).to_csv('preds3-2-cat.csv')

Learning rate set to 0.071321
0:	learn: 129.1362799	total: 8.22ms	remaining: 8.21s
1:	learn: 125.5933798	total: 15.8ms	remaining: 7.87s
2:	learn: 122.3523723	total: 23.1ms	remaining: 7.66s
3:	learn: 119.4369042	total: 29.9ms	remaining: 7.45s
4:	learn: 116.8560770	total: 36.6ms	remaining: 7.28s
5:	learn: 114.5327665	total: 43.7ms	remaining: 7.25s
6:	learn: 112.5044414	total: 50.9ms	remaining: 7.22s
7:	learn: 110.6245341	total: 57.9ms	remaining: 7.18s
8:	learn: 109.0465773	total: 64.9ms	remaining: 7.15s
9:	learn: 107.6669782	total: 72.4ms	remaining: 7.17s
10:	learn: 106.2985052	total: 79.4ms	remaining: 7.14s
11:	learn: 105.0653009	total: 86.9ms	remaining: 7.15s
12:	learn: 103.9212867	total: 94.4ms	remaining: 7.16s
13:	learn: 102.9168727	total: 102ms	remaining: 7.16s
14:	learn: 101.9804042	total: 109ms	remaining: 7.14s
15:	learn: 101.1549221	total: 116ms	remaining: 7.12s
16:	learn: 100.4138814	total: 122ms	remaining: 7.08s
17:	learn: 99.7748277	total: 130ms	remaining: 7.07s
18:	learn: 99.

173:	learn: 81.6083828	total: 1.41s	remaining: 6.68s
174:	learn: 81.5284036	total: 1.42s	remaining: 6.68s
175:	learn: 81.4481899	total: 1.43s	remaining: 6.68s
176:	learn: 81.4013014	total: 1.44s	remaining: 6.67s
177:	learn: 81.3590244	total: 1.44s	remaining: 6.66s
178:	learn: 81.3154286	total: 1.45s	remaining: 6.66s
179:	learn: 81.2578291	total: 1.46s	remaining: 6.65s
180:	learn: 81.2006290	total: 1.47s	remaining: 6.64s
181:	learn: 81.1887457	total: 1.48s	remaining: 6.63s
182:	learn: 81.1376778	total: 1.48s	remaining: 6.62s
183:	learn: 81.1111961	total: 1.49s	remaining: 6.61s
184:	learn: 81.0767744	total: 1.5s	remaining: 6.6s
185:	learn: 81.0360363	total: 1.5s	remaining: 6.59s
186:	learn: 81.0253326	total: 1.51s	remaining: 6.58s
187:	learn: 80.9805382	total: 1.52s	remaining: 6.57s
188:	learn: 80.9236173	total: 1.53s	remaining: 6.57s
189:	learn: 80.8900882	total: 1.54s	remaining: 6.56s
190:	learn: 80.8576772	total: 1.55s	remaining: 6.55s
191:	learn: 80.8172631	total: 1.55s	remaining: 6.

339:	learn: 76.2880718	total: 2.81s	remaining: 5.46s
340:	learn: 76.2683458	total: 2.82s	remaining: 5.46s
341:	learn: 76.2354874	total: 2.83s	remaining: 5.46s
342:	learn: 76.1450304	total: 2.85s	remaining: 5.45s
343:	learn: 76.1301178	total: 2.85s	remaining: 5.44s
344:	learn: 76.1157527	total: 2.86s	remaining: 5.44s
345:	learn: 76.0861118	total: 2.87s	remaining: 5.43s
346:	learn: 76.0535872	total: 2.88s	remaining: 5.42s
347:	learn: 76.0351668	total: 2.89s	remaining: 5.41s
348:	learn: 76.0236903	total: 2.9s	remaining: 5.4s
349:	learn: 75.9779944	total: 2.9s	remaining: 5.39s
350:	learn: 75.9671774	total: 2.91s	remaining: 5.39s
351:	learn: 75.9536541	total: 2.92s	remaining: 5.38s
352:	learn: 75.9321893	total: 2.93s	remaining: 5.37s
353:	learn: 75.9177065	total: 2.94s	remaining: 5.36s
354:	learn: 75.8969657	total: 2.95s	remaining: 5.36s
355:	learn: 75.8824139	total: 2.96s	remaining: 5.35s
356:	learn: 75.8470245	total: 2.97s	remaining: 5.34s
357:	learn: 75.8287930	total: 2.97s	remaining: 5.

506:	learn: 72.4889223	total: 4.23s	remaining: 4.11s
507:	learn: 72.4776298	total: 4.24s	remaining: 4.11s
508:	learn: 72.4633123	total: 4.25s	remaining: 4.1s
509:	learn: 72.4331692	total: 4.25s	remaining: 4.09s
510:	learn: 72.4236770	total: 4.26s	remaining: 4.08s
511:	learn: 72.4142985	total: 4.27s	remaining: 4.07s
512:	learn: 72.4028904	total: 4.28s	remaining: 4.06s
513:	learn: 72.3790786	total: 4.29s	remaining: 4.05s
514:	learn: 72.3414123	total: 4.29s	remaining: 4.04s
515:	learn: 72.3303745	total: 4.3s	remaining: 4.03s
516:	learn: 72.3252800	total: 4.31s	remaining: 4.02s
517:	learn: 72.3166897	total: 4.31s	remaining: 4.01s
518:	learn: 72.2727393	total: 4.32s	remaining: 4s
519:	learn: 72.2658654	total: 4.33s	remaining: 4s
520:	learn: 72.2483951	total: 4.34s	remaining: 3.99s
521:	learn: 72.2308672	total: 4.34s	remaining: 3.98s
522:	learn: 72.2109785	total: 4.35s	remaining: 3.97s
523:	learn: 72.1878580	total: 4.36s	remaining: 3.96s
524:	learn: 72.1732050	total: 4.37s	remaining: 3.95s
5

662:	learn: 69.8726503	total: 5.66s	remaining: 2.88s
663:	learn: 69.8688789	total: 5.67s	remaining: 2.87s
664:	learn: 69.8552983	total: 5.68s	remaining: 2.86s
665:	learn: 69.8443316	total: 5.69s	remaining: 2.85s
666:	learn: 69.8363472	total: 5.7s	remaining: 2.85s
667:	learn: 69.8228282	total: 5.71s	remaining: 2.84s
668:	learn: 69.8034719	total: 5.71s	remaining: 2.83s
669:	learn: 69.7894858	total: 5.72s	remaining: 2.82s
670:	learn: 69.7767573	total: 5.73s	remaining: 2.81s
671:	learn: 69.7658415	total: 5.74s	remaining: 2.8s
672:	learn: 69.7576328	total: 5.75s	remaining: 2.79s
673:	learn: 69.7437838	total: 5.76s	remaining: 2.79s
674:	learn: 69.7399163	total: 5.77s	remaining: 2.78s
675:	learn: 69.6923044	total: 5.78s	remaining: 2.77s
676:	learn: 69.6700142	total: 5.79s	remaining: 2.76s
677:	learn: 69.6525797	total: 5.79s	remaining: 2.75s
678:	learn: 69.6399680	total: 5.8s	remaining: 2.74s
679:	learn: 69.6236985	total: 5.81s	remaining: 2.73s
680:	learn: 69.6151277	total: 5.82s	remaining: 2.

838:	learn: 67.4923334	total: 7.07s	remaining: 1.36s
839:	learn: 67.4822365	total: 7.08s	remaining: 1.35s
840:	learn: 67.4745051	total: 7.09s	remaining: 1.34s
841:	learn: 67.4632989	total: 7.1s	remaining: 1.33s
842:	learn: 67.4485603	total: 7.11s	remaining: 1.32s
843:	learn: 67.4334183	total: 7.12s	remaining: 1.31s
844:	learn: 67.4240080	total: 7.13s	remaining: 1.31s
845:	learn: 67.4087899	total: 7.13s	remaining: 1.3s
846:	learn: 67.3839364	total: 7.14s	remaining: 1.29s
847:	learn: 67.3751490	total: 7.15s	remaining: 1.28s
848:	learn: 67.3521106	total: 7.16s	remaining: 1.27s
849:	learn: 67.3333899	total: 7.17s	remaining: 1.26s
850:	learn: 67.3171474	total: 7.18s	remaining: 1.26s
851:	learn: 67.3034922	total: 7.19s	remaining: 1.25s
852:	learn: 67.2890325	total: 7.2s	remaining: 1.24s
853:	learn: 67.2801923	total: 7.21s	remaining: 1.23s
854:	learn: 67.2590838	total: 7.21s	remaining: 1.22s
855:	learn: 67.2488805	total: 7.22s	remaining: 1.22s
856:	learn: 67.2273050	total: 7.23s	remaining: 1.